In [92]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

logs_path = os.path.join("C:","Users","alixd","Documents","CPE","5A","projet_majeure","S6_G6_Deleule_Delzenne-Zamparutti","IMU","logs")
logs_path = os.path.abspath("logs")

log_files = [os.path.join(logs_path,f) for f in os.listdir(logs_path)]
log_files

['c:\\Users\\alixd\\Documents\\CPE\\5A\\projet_majeure\\S6_G6_Deleule_Delzenne-Zamparutti\\IMU\\logs\\chest_tap.csv',
 'c:\\Users\\alixd\\Documents\\CPE\\5A\\projet_majeure\\S6_G6_Deleule_Delzenne-Zamparutti\\IMU\\logs\\d_shake_left.csv',
 'c:\\Users\\alixd\\Documents\\CPE\\5A\\projet_majeure\\S6_G6_Deleule_Delzenne-Zamparutti\\IMU\\logs\\d_shake_right.csv',
 'c:\\Users\\alixd\\Documents\\CPE\\5A\\projet_majeure\\S6_G6_Deleule_Delzenne-Zamparutti\\IMU\\logs\\other.csv',
 'c:\\Users\\alixd\\Documents\\CPE\\5A\\projet_majeure\\S6_G6_Deleule_Delzenne-Zamparutti\\IMU\\logs\\saturation_test.csv',
 'c:\\Users\\alixd\\Documents\\CPE\\5A\\projet_majeure\\S6_G6_Deleule_Delzenne-Zamparutti\\IMU\\logs\\stationnary.csv',
 'c:\\Users\\alixd\\Documents\\CPE\\5A\\projet_majeure\\S6_G6_Deleule_Delzenne-Zamparutti\\IMU\\logs\\test_d_shake.csv']

In [93]:
import numpy as np
import matplotlib.pyplot as plt

def detect_shakes_with_derivative(accel_data, sampling_rate=52, threshold=1.5):
    """
    Detect shakes from accelerometer data using the derivative and zero crossings.
    
    Parameters:
        accel_data (list or np.array): Smoothed buffer of accelerometer norm values (1 second of data).
        sampling_rate (int): Sampling rate in Hz (default: 52 Hz).
        threshold (float): Minimum value to qualify as a peak (e.g., 1.5 m/s²).
        
    Returns:
        list: Indices of detected peaks.
    """
    # Calculate the derivative of the acceleration data
    derivative = np.diff(accel_data)
    
    # Find zero crossings (where the derivative changes sign)
    zero_crossings = np.where(np.diff(np.sign(derivative)))[0]
    
    # Check for peaks: derivative changes from positive to negative
    peaks = [
        idx+1 for idx in zero_crossings
        if derivative[idx] > 0 and derivative[idx + 1] < 0 and accel_data[idx] > threshold
    ]
    
    return peaks

def plot_peaks(accel_data, peaks, sampling_rate=52):
    """
    Plot accelerometer data and highlight detected peaks.
    
    Parameters:
        accel_data (list or np.array): Smoothed buffer of accelerometer norm values.
        peaks (list): Indices of detected peaks.
        sampling_rate (int): Sampling rate in Hz (default: 52 Hz).
    """
    time = np.arange(len(accel_data)) / sampling_rate  # Time array for x-axis
    
    plt.plot(time, accel_data, label="Accelerometer Norm", color="blue")
    plt.scatter(
        [time[p] for p in peaks],
        [accel_data[p] for p in peaks],
        color="red",
        label="Detected Peaks",
        zorder=5,
    )
    # plt.axhline(y=threshold, color="green", linestyle="--", label="Threshold")
    plt.title("Accelerometer Data with Detected Peaks")
    plt.xlabel("Time (s)")
    plt.ylabel("Acceleration (m/s²)")
    plt.legend()
    plt.grid(True)
    plt.show()

In [94]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.signal import butter, filtfilt

def butter_lowpass_filter(data, cutoff, fs, order=4):
    """
    Apply a Butterworth low-pass filter to the data.
    
    Parameters:
        data (array): Input data to filter.
        cutoff (float): Cutoff frequency of the filter in Hz.
        fs (float): Sampling frequency in Hz.
        order (int): Order of the filter.

    Returns:
        array: Filtered data.
    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def plot(filepath):
    df = pd.read_csv(filepath, encoding='utf-8', skiprows=1)

    df = df[:53]

    # Convert time from microseconds to seconds
    df['time[s]'] = df['time[us]'] / 1e6
    time_s = df['time[s]']

    # Sampling frequency (assumes evenly spaced data)
    fs = 52
    cutoff = 10  # Low-pass filter cutoff frequency in Hz

    # Compute norms for accelerometer and gyroscope data
    acc_norm = np.sqrt(df['acc_x[mg]']**2 + df['acc_y[mg]']**2 + df['acc_z[mg]']**2)
    gyro_norm = np.sqrt(df['gyro_x[mdps]']**2 + df['gyro_y[mdps]']**2 + df['gyro_z[mdps]']**2)

    # Apply low-pass filter to norms
    acc_norm_filtered = butter_lowpass_filter(acc_norm, cutoff, fs)
    gyro_norm_filtered = butter_lowpass_filter(gyro_norm, cutoff, fs)

    # Compute derivatives of the filtered norms
    acc_norm_derivative = np.gradient(acc_norm_filtered, time_s)
    gyro_norm_derivative = np.gradient(gyro_norm_filtered, time_s)

    # Plot Acceleration Data (acc_x, acc_y, acc_z) and their derivative
    plt.figure(figsize=(16, 12))
    plt.suptitle(os.path.basename(filepath))

    # First subplot: Accelerometer data
    plt.subplot(2, 2, 1)
    plt.plot(time_s, df['acc_x[mg]'], label='Acc X')
    plt.plot(time_s, df['acc_y[mg]'], label='Acc Y')
    plt.plot(time_s, df['acc_z[mg]'], label='Acc Z')
    plt.plot(time_s, acc_norm, label='Acc Norm', linestyle='--', color='black')
    plt.plot(time_s, acc_norm_filtered, label='Filtered Acc Norm', linestyle='--', color='green')
    
    plt.title('Accelerometer Data')
    plt.xlabel('Time [s]')
    plt.ylabel('Acceleration [mg]')
    plt.legend()
    plt.grid(True)

    # Second subplot: Derivative of accelerometer norm
    plt.subplot(2, 2, 2)

    sampling_rate = 52
    threshold = 15e3  # Set a threshold based on your data's typical range
    peaks = detect_shakes_with_derivative(acc_norm_derivative, sampling_rate, threshold)
    print(f"Number of shakes detected: {len(peaks)}")
    # Plot the accelerometer data and detected peaks
    plot_peaks(acc_norm_derivative, peaks, sampling_rate)

    # plt.plot(time_s, acc_norm_derivative, label='d(Filtered Acc Norm)/dt', color='red')
    plt.title('Derivative of Accelerometer Norm (Filtered)')
    plt.xlabel('Time [s]')
    plt.ylabel('Derivative [mg/s]')
    plt.legend()
    plt.grid(True)

    # Third subplot: Gyroscope data
    plt.subplot(2, 2, 3)
    plt.plot(time_s, df['gyro_x[mdps]'], label='Gyro X')
    plt.plot(time_s, df['gyro_y[mdps]'], label='Gyro Y')
    plt.plot(time_s, df['gyro_z[mdps]'], label='Gyro Z')
    plt.plot(time_s, gyro_norm, label='Gyro Norm', linestyle='--', color='black')
    plt.plot(time_s, gyro_norm_filtered, label='Filtered Gyro Norm', linestyle='--', color='green')
    plt.title('Gyroscope Data')
    plt.xlabel('Time [s]')
    plt.ylabel('Gyroscope [mdps]')
    plt.legend()
    plt.grid(True)

    # Fourth subplot: Derivative of gyroscope norm
    plt.subplot(2, 2, 4)
    
    sampling_rate = 52
    threshold = 1.5e6  # Set a threshold based on your data's typical range
    peaks = detect_shakes_with_derivative(gyro_norm_derivative, sampling_rate, threshold)
    print(f"Number of shakes detected: {len(peaks)}")
    # Plot the accelerometer data and detected peaks
    plot_peaks(gyro_norm_derivative, peaks, sampling_rate)

    # plt.plot(time_s, gyro_norm_derivative, label='d(Filtered Gyro Norm)/dt', color='orange')
    plt.title('Derivative of Gyroscope Norm (Filtered)')
    plt.xlabel('Time [s]')
    plt.ylabel('Derivative [mdps/s]')
    plt.legend()
    plt.grid(True)

    # Adjust layout and show plots
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust title margin
    plt.show()


In [95]:
%matplotlib qt
for f in log_files:
    plot(f)

Number of shakes detected: 1
Number of shakes detected: 1
Number of shakes detected: 2
Number of shakes detected: 4
Number of shakes detected: 2
Number of shakes detected: 2
Number of shakes detected: 0
Number of shakes detected: 0
Number of shakes detected: 0
Number of shakes detected: 0
Number of shakes detected: 0
Number of shakes detected: 0
Number of shakes detected: 0
Number of shakes detected: 0
